In [20]:
import torch
torch.cuda.get_device_name(0)

'GeForce 940MX'

In [21]:
torch.cuda.is_available()

True

In [3]:
# import numpy as np
import pandas as pd

In [9]:
data = pd.read_csv('iris.csv')
data.shape

(150, 6)

In [10]:
data.drop('Id', inplace=True, axis=1)
data.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [23]:
train_set, test_set = torch.utils.data.random_split(data, [120, 30])

In [26]:
train_set

In [27]:
test_set

In [ ]:
# model = nn.Sequential(
#           nn.Conv2d(1,20,5),
#           nn.ReLU(),
#           nn.Conv2d(20,64,5),
#           nn.ReLU()
#         )
